#**Data visualization**

Data visualization is the representation of data through use of common graphics, such as charts, plots, infographics, and even animations.

These visual displays of information communicate complex data relationships and data-driven insights in a way that is easy to understand.

The main goal of data visualization is to make it easier to identify patterns, trends and outliers in large data sets.

Data visualization is the representation of data through use of common graphics, such as charts, plots, infographics, and even animations.

These visual displays of information communicate complex data relationships and data-driven insights in a way that is easy to understand.

The main goal of data visualization is to make it easier to identify patterns, trends and outliers in large data sets.


**Advantages:**

* Easily sharing information.

* Interactively explore opportunities.

* Visualize patterns and relationships.

* Quickly see trends and outliers.

In [ ]:
#These are libraries that you should import if needed for your notebook.
#import libraries
import pandas as pd
import seaborn as sns
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
from google.colab import files

## Visualisation

In [ ]:
# upload file to Colab
uploaded = files.upload()

In [ ]:
# read the file into a variable
olympics = pd.read_csv('olympics2024.csv')

In [ ]:
# rename 'GDP per capita (current US$)'
olympics = olympics.rename(columns={'GDP per capita (current US$)' : 'GDPpc' })
olympics


# Scatter Plot

In [ ]:
# use a scatter plot to see if there is a relationship between a country's
# polulation size and the number of gold medals the country obtained



In [ ]:
# zoom in to the countries with less than 400mil people



Draw a scatter plot with Population on the x-axis, gold on the y-axis with countries from different continents having different colours.

In [ ]:
# make use of Seaborn


Let's determine visually what the relation is (if any) between GDP per capita and the number of medals obtained, highlighting each continent in a different colour.

In [ ]:
# make use of Seaborn


# moves legend to right



# Histogram

In [ ]:
# Draw a histogram of Total medals


In [ ]:
# Create a histogram with Seaborn


# Bar Graph

In [ ]:
# calculate the average medals of each type (gold, silver, bronze) obtained by
# each contiment
medal_aves = olympics.pivot_table(index='Continent', values=['Gold','Silver','Bronze'], aggfunc='mean')[['Gold','Silver','Bronze']]
medal_aves

In [ ]:
# create a barr graph


In [ ]:
# create a horizontal barr graph


# Pie Chart

Looking at only the European countries. Let's draw a pie chart of the percentage of gold medals each European country has obtained.

In [ ]:
# geneate a table with only the European countries gold medal counts
eu_gold = olympics.where(olympics['Continent'] == 'Europe').where(olympics['Gold'] > 0).dropna().sort_values('Gold', ascending=False)[['Country', 'Gold']]
eu_gold

In [ ]:
# create pie chart



Draw a pie chart of gold medals obtained where countries with less than 4 gold medals are group together in a single catagory (in a single pie section).

In [ ]:
# group countries with less than 4 gold medals together
eu_gold_less4 = olympics.where(olympics['Continent'] == 'Europe').where(olympics['Gold'] < 4).dropna().sort_values('Gold', ascending=False)[['Country', 'Gold']]
# create a df with a summary of this data
eu_gold_less4_grouped = pd.DataFrame([['Other',eu_gold_less4['Gold'].sum()]],columns=['Country', 'Gold'])

# create a df with a subset of data with Euro countries with 4+ gold medals
eu_gold_min4 = olympics.where(olympics['Continent'] == 'Europe').where(olympics['Gold'] >= 4).dropna().sort_values('Gold', ascending=False)[['Country', 'Gold']]


In [ ]:
# combine the dataframes
eu_gold2 = pd.concat([eu_gold_min4, eu_gold_less4_grouped])
eu_gold2

In [ ]:
# create pie chart
plots.figure(figsize=(8, 8))  # Adjusting the figure size for better visualization
plots.pie(eu_gold2['Gold'], labels=eu_gold2['Country'], autopct='%1.1f%%', startangle=90)

# Spider Chart \ Radar Chart:

Visualize data across multiple categories for several groups.

In [ ]:
# collect top 5 gold medal winning EU countries info
top5 = olympics.where(olympics['Continent'] == 'Europe').dropna().sort_values('Gold', ascending=False).head(5)
top5

In [ ]:
# create spider chart of top5's different medal types

top5=top5[['Country','Gold','Silver','Bronze']]

# Transpose the DataFrame so that countries are the axes
top5_transposed = top5.set_index('Country').T

# Set the catagories
categories = top5_transposed.columns.tolist()

# Compute the angle of each axis
angles = np.linspace(0, 2 * np.pi, 5, endpoint=False).tolist()

# The plot is a circle, so we need to "complete the loop" and append the start to the end.
angles += angles[:1]

# Initialize the spider plot
fig, ax = plots.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

# Loop to plot each medal category
for category in top5_transposed.index:
    values = top5_transposed.loc[category].tolist()
    values += values[:1]  # Repeat the first value to close the circle
    ax.plot(angles, values, label=category)
    ax.fill(angles, values, alpha=0.25)  # Fill the area under the plot

# Labels for the countries
ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)

# Setting the range for the radial axis
ax.set_rlabel_position(30)
plots.yticks([5, 10, 15, 20, 25], ["5", "10", "15", "20", "25"], color="grey", size=10)
plots.ylim(0, top5_transposed.max().max() + 5)

# Title and legend
plots.title('Medal Counts by Category', size=20, color='darkblue', y=1.1)
ax.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))


# Sankey Plot

In [ ]:
# filter the olympics data
olympics_sankey = olympics.where(olympics['Total'] > 4).dropna()

In [ ]:
# import libraries
import plotly.graph_objects as go
import numpy as np
import colorsys

# Medal categories in desired order
categories = ['Gold', 'Silver', 'Bronze']

# Sorting continents and countries alphabetically
continents = sorted(olympics_sankey['Continent'].unique())
countries = sorted(olympics_sankey['Country'])

# Define colours for medal categories
medal_colors = {'Gold': 'gold', 'Silver': 'silver', 'Bronze': '#cd7f32'}

# Generate random colours for continents
np.random.seed(0)  # For reproducibility
continent_colors = {cont: f'#{np.random.randint(0, 256):02x}{np.random.randint(0, 256):02x}{np.random.randint(0, 256):02x}' for cont in continents}

# Function to generate shades of a colour
def generate_shades(base_color, num_shades):
    # Convert hex to RGB
    base_color = base_color.lstrip('#')
    r, g, b = int(base_color[0:2], 16), int(base_color[2:4], 16), int(base_color[4:6], 16)

    # Convert RGB to HSV
    h, s, v = colorsys.rgb_to_hsv(r/255., g/255., b/255.)

    # Generate shades by adjusting value (brightness)
    shades = []
    for i in range(num_shades):
        shade_value = v * (1 - (i / num_shades))
        r, g, b = colorsys.hsv_to_rgb(h, s, shade_value)
        r, g, b = int(r * 255), int(g * 255), int(b * 255)
        shades.append(f'rgba({r},{g},{b},1)')

    return shades

# Define shades for countries based on continent colours
country_colors = {}
for continent in continents:
    # Generate shades for countries within the continent
    continent_countries = olympics_sankey.loc[olympics_sankey['Continent'] == continent, 'Country']
    shades = generate_shades(continent_colors[continent], len(continent_countries))
    country_colors.update(dict(zip(continent_countries, shades)))

# Function to adjust colour transparency
def adjust_color_transparency(color, alpha):
    if color.startswith('#'):
        # Convert hex to RGB
        r = int(color[1:3], 16)
        g = int(color[3:5], 16)
        b = int(color[5:7], 16)
        return f'rgba({r},{g},{b},{alpha})'
    elif color.startswith('rgb'):
        # Extract RGB values from rgb format
        rgb_values = color[4:-1].split(',')
        return f'rgba({rgb_values[0]},{rgb_values[1]},{rgb_values[2]},{alpha})'
    else:
        # Handle named colours by converting them to RGB if possible
        named_colors = {
            'gold': '255,215,0',
            'silver': '192,192,192',
            'bronze': '205,127,50'
        }
        if color.lower() in named_colors:
            rgb_values = named_colors[color.lower()]
            return f'rgba({rgb_values},{alpha})'
        else:
            return f'{color},{alpha})'

# Create lists to hold the nodes and links
labels = []
sources = []
targets = []
values = []
link_colors = []

# Adding medal categories as nodes in specified order
labels.extend(categories)
category_colors = [medal_colors[cat] for cat in categories]

# Adding sorted continents as nodes
labels.extend(continents)
continent_colors = [continent_colors[cont] for cont in continents]

# Adding sorted countries as nodes
labels.extend(countries)
country_colors = [country_colors[country] for country in countries]

# Create links between medal categories and continents with transparency
for i, category in enumerate(categories):
    for j, continent in enumerate(continents):
        # Sum the values for each continent and medal category
        value = olympics_sankey.loc[olympics_sankey['Continent'] == continent, category].sum()
        sources.append(i)
        targets.append(len(categories) + j)
        values.append(value)
        # Use more transparency for continent links
        link_colors.append(adjust_color_transparency(medal_colors[category], 0.5))

# Create links between continents and countries with even more transparency
for j, continent in enumerate(continents):
    for k, country in enumerate(countries):
        # Find the original index of the country in the unsorted list
        if olympics_sankey.loc[olympics_sankey['Country'] == country, 'Continent'].values[0] == continent:
            for i, category in enumerate(categories):
                sources.append(len(categories) + j)
                targets.append(len(categories) + len(continents) + k)
                values.append(olympics_sankey.loc[olympics_sankey['Country'] == country, category].values[0])
                # Use even more transparency for country links
                link_colors.append(adjust_color_transparency(medal_colors[category], 0.2))

# Create Sankey diagram
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color=category_colors + continent_colors + country_colors  # Node colors
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color=link_colors  # Link colors with transparency
    )
))

# Adding title
fig.update_layout(title_text="Sankey Diagram of Medal Distribution", font_size=10)

# Show the diagram
fig.show()
